In [101]:
import os
from pathlib import Path
import json
from dotenv import load_dotenv

from google.oauth2 import service_account
from pydantic_ai import Agent, BinaryContent
from pydantic_ai.models.google import GoogleModel
from pydantic_ai.providers.google import GoogleProvider
from utils.BaseModel import ExtracaoOutput

load_dotenv()

True

In [94]:
credenciais_caminho = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
credenciais = json.load(open(credenciais_caminho))

In [42]:

nota_fiscal = Path('NFSe.pdf')
nota_fiscal

WindowsPath('NFSe.pdf')

In [ ]:
credentials = service_account.Credentials.from_service_account_file(
    credenciais_caminho,
    scopes=['https://www.googleapis.com/auth/cloud-platform'],
)
provider = GoogleProvider(credentials=credentials, project=credenciais['project_id'])
model = GoogleModel('gemini-2.5-flash', provider=provider)

system_prompt= """
Você é um agente de IA especialista em processamento de documentos fiscais brasileiros. Sua principal tarefa é analisar o conteúdo de uma Nota Fiscal de Serviço em formato de documento (PDF), extrair informações cruciais e retorná-las em um formato estruturado, de acordo com o modelo Pydantic fornecido.

**Instruções Gerais:**

1.  **Entrada:** Você receberá um documento, que é uma representação de uma Nota Fiscal de Serviço. Utilize o recurso de `Document Input` para ler e interpretar diretamente o conteúdo textual do documento.
2.  **Objetivo:** Seu objetivo é preencher de forma precisa e completa o modelo Pydantic `ExtracaoOutput` com os dados extraídos do documento.
3.  **Contexto:** A Nota Fiscal de Serviço é um documento oficial que registra a prestação de serviços. Preste muita atenção aos seguintes campos:
    * **Descrição do Serviço:** O detalhamento do serviço que foi executado.
    * **Valor do Serviço:** O valor a ser pago pelo serviço prestado.
    * **Número da Nota:** O número da nota fiscal.
    * **Data de emissão:** A data em que a nota fiscal foi emitida.
    * **Valor Total:** O valor total da nota fiscal (bruto).
    * **CNPJ do Prestador:** CNPJ de quem emitiu a nota.

**Regras de Extração:**

* **Precisão é fundamental:** Extraia os valores exatamente como aparecem no documento. Não invente ou infira informações que não estão presentes.
* **Formatos:**
    * Para valores monetários, extraia o número e formate-o como um `float` (ex: "R$ 1.500,50" deve ser extraído como `1500.50`).
    * Para CNPJ e CPF, extraia apenas os números.
    * Para datas, siga o formato `YYYY-MM-DD`. Se a data no documento estiver em outro formato (ex: DD/MM/YYYY), faça a conversão.
* **Ambiguidade:** Se um campo não for encontrado ou se a informação for ambígua, retorne `None` para aquele campo específico. Não tente adivinhar.
* **Foco no Conteúdo:** Analise todo o texto extraído do documento para localizar as informações. Ignore elementos de layout, como logos ou tabelas, e foque nos rótulos e nos dados textuais.

**Exemplo de Saída:**
```
{
    "descricao_servico": "Consultoria em TI",
    "valor_servico": 1500.00,
    "numero_nf": 123456,
    "data_emissao": "2023-10-01",
    "valor_total": 1500.00,
    "cnpj": "12345678000195"
}
"""

agent = Agent(
    model=model,
    system_prompt=system_prompt,
    output_type=ExtracaoOutput
)

In [63]:
resultado = await agent.run(
    [BinaryContent(nota_fiscal.read_bytes(), media_type='application/pdf')]
)

In [92]:

resultado.output.model_dump()
resultado_json = json.dumps(resultado.output.model_dump(), default=str, ensure_ascii=False)
print(resultado.output.model_dump())
print(resultado_json)

{'descricao_servico': 'MENSALIDADE EAD - EDUCAÇÃO SUPERIOR - GRADUAÇÃO E PÓS-GRADUAÇÃO - Ensino regular pré escolar, fundamental, médio e superior.', 'valor_servico': 138.33, 'numero_nf': 4365344, 'data_emissao': datetime.date(2025, 10, 2), 'valor_total': 138.33, 'cnpj': '04986320003139'}
{"descricao_servico": "MENSALIDADE EAD - EDUCAÇÃO SUPERIOR - GRADUAÇÃO E PÓS-GRADUAÇÃO - Ensino regular pré escolar, fundamental, médio e superior.", "valor_servico": 138.33, "numero_nf": 4365344, "data_emissao": "2025-10-02", "valor_total": 138.33, "cnpj": "04986320003139"}


In [93]:
# Cria a pasta se não existir
os.makedirs('registro_de_notas', exist_ok=True)

# Monta o nome do arquivo conforme solicitado
nome_arquivo = f"{resultado.output.data_emissao}-{resultado.output.cnpj}-{resultado.output.numero_nf}.json"
caminho_arquivo = os.path.join('registro_de_notas', nome_arquivo)

# Salva o resultado_json no arquivo
with open(caminho_arquivo, 'w', encoding='utf-8') as f:
    f.write(resultado_json)